# Biosound data - Tidy up 
Prep the biosound data for use in the dashboard

To run this in Pycharm you need to set up a kernel that points to the venv:
`python -m ipykernel install --user --name=venv-osa`

Also please note that the data files and folders in this notebook point to a specific path a local Google Drive shared folder. This will not work for others - the paths will need updating for each person.

In [ ]:
import pandas as pd
import numpy as np

import data_wrangler as dw
import os

In [ ]:
# Set up some initial things
# Local path to the main "Processed Data" folder
local_data_folder = ('/Users/michelle/Library/CloudStorage/GoogleDrive-michelle@waveformanalytics.com/.shortcut'
                     '-targets-by-id/1QNPk7Z3Yb2uhHsHCf49pw7k_2-ivhIJg/Processed_Data')

# config_file = os.path.join(local_data_folder, 'BioSound_Datasets_MapApp.csv')
config_file = os.path.join(local_data_folder, 'BioSound_Datasets_MapApp_v1.csv')
df_config = pd.read_csv(config_file)

# Folder containing annotations data files
annotations_folder = os.path.join(local_data_folder, 'Annotations')

# Load the fish/annotations codes lookup table
df_codes = pd.read_csv("../shiny/data/fish_codes.csv")

# Satellite water class data
seascaper_folder = os.path.join(local_data_folder,"All_SeascapeR")

# Output file path for the duckdb file 
db_file = os.path.join(local_data_folder, "mbon.duckdb")

In [ ]:
# Take a look at the contents of the config file
df_config

## Annotations data

### Prep Key West annotations
Key west annotations are a bit different from the others. They're in a folder that contains several .txt files that need to first be merged and then copied into the same annotations folder as the other datasets.

In [ ]:
KW_ANNOTATIONS_FOLDER = os.path.join(local_data_folder, 'Annotations/key-west-original')
KW_ANNOTATIONS_OUTFILE = os.path.join(local_data_folder, 'Annotations/KW_Annotations.csv')

# Run the function from data_wrangler to convert to the regular annotations format and move the file to the same location as the rest of the annotation files
dw.annotation_prep_kw_style(KW_ANNOTATIONS_FOLDER, KW_ANNOTATIONS_OUTFILE)

### Prep May River annotations data

May River annotations are also unique. They're stored in wide format in the "Master_Manual....xlsx" file. In this section we re-format so that it looks more like the Key West annotations and all the ship annotations (long-format).

In [ ]:
MR_ANNOTATIONS_FILE = os.path.join(local_data_folder, 'Annotations/may-river-original/Master_Manual_14M_2h_011119_071619.xlsx')
MR_ANNOTATIONS_OUTFILE = os.path.join(local_data_folder, 'Annotations/MR_Annotations.csv')

dw.annotation_prep_mr_style(MR_ANNOTATIONS_FILE, MR_ANNOTATIONS_OUTFILE, df_codes)

## Index data
Prepare and load the data from the various acoustic index files and combine them all into a big dataframe. Also create a second dataframe that has all the same index values but normalized. 

In [ ]:
# INDEX_DATA_FOLDER = os.path.join(local_data_folder, 'Revised_Indices_AllData_v2')
INDEX_DATA_FOLDER = os.path.join(local_data_folder, 'Original_Indices_AllData_v1')

df_aco0 = dw.prep_index_data(INDEX_DATA_FOLDER, normalize=False)
df_aco_norm0 = dw.prep_index_data(INDEX_DATA_FOLDER, normalize=True)

In [ ]:
# Set time zones to local time
dw.update_time_zone(df_aco0, df_config)
dw.update_time_zone(df_aco_norm0, df_config)

## Add the annotation information to index dataframes

Add new columns to the index dataframes for all possible indices for both presence and count.

In [ ]:
unique_codes = np.unique(df_codes['code']).tolist()

# Add new columns that are named using unique_fish_codes. These will become the 
df_aco = dw.add_new_columns(df_aco0, unique_codes)
df_aco_norm = dw.add_new_columns(df_aco_norm0, unique_codes)

In [ ]:
df_aco_anno = dw.add_annotations_to_df(df_aco, df_config, df_codes, annotations_folder)
df_aco_norm_anno = dw.add_annotations_to_df(df_aco_norm, df_config, df_codes, annotations_folder)

## Prep Seascaper Data


In [ ]:
df_seascaper = dw.prep_seascaper_data(seascaper_folder, df_config)

## Create DuckDB

Save the main pandas dataframes to a duckdb file.

In [ ]:
# Prep the dataframe list for export
df_dict = {
    "t_aco2": df_aco_anno,
    "t_aco_norm2": df_aco_norm_anno,
    "t_seascaper": df_seascaper
}

dw.duckdb_export(db_file, df_dict)

In [ ]:
np.shape(df_aco_anno)

In [ ]:
db_file